---
# Configurando a instância do curso
---

## Criando uma instância no AWS

Ao invés de utilizar a imagem da Amazon, vamos usar o Ubuntu 16.04 LTS.

Para criar uma nova instância basta seguir os passos explicitados [aqui](/notebooks/dev/2017-01-15-MSS-Configurando-AWS.ipynb#segundo-passo), tomando o cuidado de:

* escolher a AMI com *Ubuntu Server 16.04 LTS (HVM), SSD Volume Type - ami-b7a114d7*.
* aumentar o espaço de armazenamento seguindo os seguintes passos na tela de **Review and Launch**:
    * Selecionar **Edit storage**;
    * Remover o volume *Instance Store 0*;
    * Aumentar o espaço do volume *Root* para 30 GiB, por exemplo.
    * Selecionar **Review and Launch**;

Depois de criada e conectada a instância, realizar o acesso SSH para fazer a instalação dos pacotes necessários:

* ``ssh -i "<path_arquivo_chave_pem>" ubuntu@<dns_público_instância>``

## Atualizando a instância

Atualização do sistema e instalação de pacotes extras:
* ``sudo apt-get update && sudo apt-get upgrade``
* ``sudo apt-get install curl linux-image-extra-$(uname -r) linux-image-extra-virtual``
   
Instalação do Docker:
* ``sudo apt-get install apt-transport-https ca-certificates``
* ``curl -fsSL https://yum.dockerproject.org/gpg | sudo apt-key add -``
* ``apt-key fingerprint 58118E89F3A912897C070ADBF76221572C52609D``
* ``sudo add-apt-repository "deb https://apt.dockerproject.org/repo/ ubuntu-$(lsb_release -cs) main"``
* ``sudo apt-get update``
* ``sudo apt-get -y install docker-engine``

Instalação do driver da placa de vídeo:
* ``sudo add-apt-repository ppa:graphics-drivers/ppa``
* ``sudo apt-get update``
* ``sudo apt-get install nvidia-361``
* ``sudo apt-get install nvidia-modprobe``

Download da versão mais recente do nvidia-docker (precisa ser a 1.0.0 final para ser compatível com o docker 1.13)
* ``wget -P /tmp https://github.com/NVIDIA/nvidia-docker/releases/download/v1.0.0/nvidia-docker_1.0.0-1_amd64.deb``
* ``sudo dpkg -i /tmp/nvidia-docker*.deb && rm /tmp/nvidia-docker*.deb``

Teste do nvidia-docker, deverá baixar a imagem e ao ser executado, encontrar a(s) placa(s) de vídeo da máquina:
* ``sudo nvidia-docker run --rm nvidia/cuda nvidia-smi``

Mudança de permissão para permitir uso do docker sem usuário root:
* ``sudo usermod -a -G docker ubuntu``

Para que as permissões sejam atualizadas é necessário reiniciar o sistema:
* ``sudo reboot``

**Atenção**: Ao reiniciar o sistema o serviço nvidia-docker-plugin não está inicializando automaticamente. Isto pode ser verificado com o comando:

* ``service nvidia-docker status``

Caso não esteja mesmo rodando é preciso inicializá-lo com:

* ``sudo service nvidia-docker start``

**Próximo passo**: Realizar esta inicialização automaticamente.

## Baixando e executando a instância a ser utilizada no curso

Para executar o Docker do curso inicialmente é preciso criar as pastas para as quais os notebooks e datasets serão mapeados:

Para voltar ao diretório home do usuário, caso não esteja no mesmo:
* ``cd ~``

Para criar as pastas:
* ``mkdir notebooks datasets``

Então poderá ser rodada a instância do curso. Se ela ainda não estiver sido baixada, isto será feito automaticamente:
* ``nvidia-docker run --rm --name jupyter -it -p 8888:8888 -v /home/ubuntu/notebooks:/opt/projects \
-v /home/ubuntu/datasets:/opt/datasets rcm2016/lasagne_cuda:1.2``

## Referências

* Passo a passo para instalação do nvidia-docker: http://seanbe.github.io/post/deep-learning-aws/
* Instalação do Docker: https://docs.docker.com/engine/installation/linux/ubuntu/